In [1]:
%run library_TS.py

Python version: 3.9.16 (main, Mar  8 2023, 10:39:24) [MSC v.1916 64 bit (AMD64)]
NumPy version: 1.23.5
pandas version: 1.5.3
matplotlib: 3.7.1
TSfresh: 0.20.0
Sklearn version: 1.1.3
Xgboost version: 1.5.0
Lightgbm version: 2.2.3


c:\Users\Windows\miniconda3\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


# DELETES THE FIRST LINES OF THE TXT FILES

In [2]:
def delete_lines(p) : 
    """Delete only the first useless lines of the txt file (like patient's name, variable's name, line break...)"""
    path = p # Definition of the path containing all the files
    files = os.listdir(path) # Creation of the list containing all the files name
    os.chdir(path) # Defintion of the path as a current path
    for filename in files : 
        with open(filename, 'r') as f : 
        # encoding='latin-1'
            lines = f.readlines()
        with open(filename, 'w') as fp:
        # iterate each line
            for number, line in enumerate(lines):
                # delete line 0 to 5. or pass any Nth line you want to remove
                if number not in [0,1,2,3,4,5]:
                    fp.write(line)    

In [14]:
delete_lines("C:/Users/Windows/Documents/Essai/Ventilation/VT/")

# TRUNCATE TIME SERIES AND MAKE SOME STATISTICAL ANALYSIS

The files containes more lines as we like. So this function it's developped to conserve only the time interval of interest (like 2 days).

In [18]:
# Import the dataframe contening the Id and date point 
Ref = pd.read_excel("//wsl.localhost/Ubuntu/home/romain/GITHUB/Extub-IA/BDD/Target.xls", dtype={'ID': 'str'}, #ID to str, because the ID have a little different format like this : 0001, 0002
                    parse_dates=['Date_sevrage'])

# Creation of a dictionnary containing the ID and the date point
data = {
    "ID":Ref["ID"],
    "Date point": Ref["Date_sevrage"]
}
# Convert it in a Pandas DataFrame
df = pd.DataFrame(data)

# Definition of the folder containing the TS.
source_dir = "C:/Users/Windows/Documents/Essai/Ventilation/"

# Definition of the time delta for the truncation, here 2 days
td = pd.Timedelta(days=2)

# Browse the different folder and subfolder
for root, dirs, files in os.walk(source_dir):
    for dir in dirs:
        # Browse all the text file in the subfolder
        for file in os.listdir(os.path.join(root, dir)):
            if file.endswith('.txt'):
                # Complete path of the old file
                old_file = os.path.join(root, dir, file)
                # Complete path of the new file
                new_file = os.path.join(root.replace("Ventilation", "Ventilation_2H"), dir, file)
                try :
                # Modification of the contains
                    subdata = pd.read_csv(
                                old_file,
                                sep="\s+",  # columns are separated by spaces
                                header=None,  # there's no header information
                                parse_dates=[[0, 1]],  # the first and second columns should be combined and converted to datetime objects
                                #infer_datetime_format=True,
                                decimal=",",
                                #parse_dates=['0_1'],
                                dayfirst=True,
                                on_bad_lines='skip'
                            )
                    subdata["0_1"] = pd.to_datetime(subdata["0_1"], errors='coerce')
                    date = df.loc[df['ID'] == file[:4], 'Date point'].item()
                # Only if there is no missing values
                    if date != "na":
                        date = pd.to_datetime(date)
                        mask = (subdata["0_1"] > date - td) & (subdata["0_1"] <= date)
                        subdata.loc[mask].to_csv(new_file, index = False, header = False, sep='\t')
                # Add a message if the file is empty
                except EmptyDataError:
                    print("fichier vide")


